In [0]:
pip install ndlib

In [0]:
pip install cdlib

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from cdlib import algorithms
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import pandas as pd

In [0]:
def ICM(graph, per_inf, threshold, num_iter):
  model = ep.IndependentCascadesModel(graph)

  config = mc.Configuration()
  config.add_model_parameter('percentage_infected', per_inf)

  for e in graph.edges():
      config.add_edge_configuration("threshold", e, threshold) # edge attribute setting

  model.set_initial_status(config)

  iterations = model.iteration_bunch(num_iter, node_status=True)

  trends = model.build_trends(iterations)
  return iterations

  %matplotlib inline
  from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
  viz = DiffusionTrend(model, trends)
  print(viz)
  viz.plot()

In [0]:
files = ['/content/drive/My Drive/auth/SNA Project/Datasets/ca-sandi_auths.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/ca-netscience.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/bio-celegans.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/bio-diseasome.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/rt-twitter-copen.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/econ-mbeause.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/econ-wm2.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/econ-mbeacxc.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/ca-CSphd.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/ca-GrQc.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/ca-HepPh.mtx'
,'/content/drive/My Drive/auth/SNA Project/Datasets/econ-psmigr1.mtx']

In [0]:
def iter_analysis(z,length):
  count_inf = count_rem = avg_inf = avg_rem = 0
  leny = length
  for i in range(0,len(z)):
    x = z[i].get('status')
    for key in x:
      y =x[key]
      if y == 1:
        count_inf += 1
      elif y == 2:
        count_rem += 1
  avg_inf = count_inf / leny
  avg_rem = count_rem / leny
  return[count_inf, avg_inf, count_rem, avg_rem]

In [0]:
def icm_data(a, b, num, G):
  df = pd.DataFrame(columns = ['Graph', 'Model', 'Size', 'Infected Number', 'Avg_inf', 'Fin_rem', 'Avg_rem' ])
  lenG = G.order()
  inf_num = avg_inf = fin_rem = avg_rem = 0

  n = 5 
  for i in range(1,n):
    z = ICM(G, a, b, num)
    x = iter_analysis(z, lenG) 
    inf_num += x[0]
    avg_inf += x[1]
    fin_rem += x[2]
    avg_rem += x[3]

  df = df.append({'Graph' : 'Full Graph', 'Model' : 'ICM', 'Size' : lenG, 'Infected Number' :inf_num/n ,'Avg_inf' :avg_inf/n, 'Fin_rem' : fin_rem/n, 'Avg_rem' : avg_rem/n} , ignore_index=True)
  return df


In [0]:
def aggegateIC_datasets(files):  
  import seaborn as sn
  dataset_csv_file = '/content/drive/My Drive/auth/SNA Project/Datasets/SNA Datasets.csv'
  final_df = pd.DataFrame(columns = ['Model', 'Seed nodes', 'Inf_Threshold', 'Graph_size', 'FG_infection'])
  import numpy as np
  perc = np.arange(0.1, 0.4, 0.1)
  num = 10
  for a in perc:
    for b in perc:
      av = pd.DataFrame(columns = ['Model', 'Seed nodes', 'Inf_Threshold', 'Graph_size', 'FG_infection']) ### TEMPORARY TEST!!!!!!!!!!!!!!
      for f in files:
        df_csv = pd.read_csv(dataset_csv_file)
        G = nx.read_adjlist(f)
        df = icm_data(a, b, num, G) ### average of 5 experiments
        model = df.loc[0, 'Model']
        lenG = G.order()
        FG = df.loc[0,'Infected Number'] / lenG
        count = df.loc[1:,'Infected Number'].sum() / lenG
        av = av.append({'Model' : model,'Seed nodes' : a, 'Inf_Threshold' : b, 'Graph_size' : lenG, 'FG_infection' : FG} , ignore_index=True)
      av = pd.concat([av,df_csv],axis=1)
      final_df = final_df.append(av,ignore_index=True, sort=False)    
  return final_df

In [0]:
data = aggegateIC_datasets(files)
print(data)

In [0]:
final = pd.DataFrame()
for i in range(0,16):
    corrM = data.loc[(12*i):(12*i+11)].corrwith(data.loc[(12*i):(12*i+11)]['FG_infection'], method='pearson', axis = 0)
    corrM = corrM.to_frame()
    final = pd.concat([final, corrM],axis=1, ignore_index=True)
print(final.mean(axis=1).round(2))


In [0]:
import seaborn as sn
sn.heatmap(data.loc[0:11].corr(method='pearson'), annot=True, fmt=".2f")